In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TFAutoModelForMaskedLM, AutoModelForMaskedLM

model_checkpoint = "jackaduma/SecBERT"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
model.summary()

Model: "tf_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  82860288  
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  40977184  
                                                                 
Total params: 83504416 (318.54 MB)
Trainable params: 83504416 (318.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
text = "She works as a [MASK]."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> She works as a server.
>>> She works as a user.
>>> She works as a lure.
>>> She works as a journalist.
>>> She works as a whole.


In [ ]:
from datasets import load_dataset

cnn_dataset = load_dataset("cnn_dailymail","3.0.0")
cnn_dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
sample = cnn_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['highlights']}'")


'>>> Review: John and .
Audrey Cook were discovered alongside their daughter, Maureen .
They were found at Tremarle Home Park in Cornwall .
Investigators say the three died of carbon monoxide .
poisoning .'

'>>> Review: NEW: Libya can serve as example of cooperation, White House spokesman says .
Resolution calls for preventing nuclear weapons from being stolen, used by military .
Obama, Russian President Dimitry Medvedev working to reduce stockpiles .
Venezuelan president Hugo Chavez on "Larry King Live" tonight, 9 ET .'

'>>> Review: Very Reverend Robert Waddington sexually abused choirboys for decades .
Inquiry into the abuse has slammed Lord Hope, former Archbishop of York .
Report claims he was made aware of misconduct 19 times but did not act .
Despite this - he still holds influential post and sits in the House of Lords .'


In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["highlights"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_datasets = cnn_dataset.map(
    tokenize_function, batched=True, remove_columns=['article', 'highlights', 'id']
)
tokenized_datasets

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 11490
    })
})

In [ ]:
tokenizer.model_max_length

1000000000000000019884624838656

In [ ]:
chunk_size = 256

In [ ]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 57'
'>>> Review 1 length: 75'
'>>> Review 2 length: 61'


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 193'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 193'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 75854
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3943
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3243
    })
})

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'pay. vick admits funding dogfighting operation but says he did not gamble. vick due in federal court monday ; future in nfl remains uncertain. [SEP] [CLS] parents beam with pride, can\'t stop from smiling from outpouring of support. mom : " i was so happy i didn\'t know what to do " burn center in u. s. has offered to provide treatment for reconstructive surgeries. dad says, " anything for youssif " [SEP] [CLS] aid workers : violence, increased cost of living drive women to prostitution. group is working to raise awareness of the problem with iraq\'s political leaders. two iraqi mothers tell cnn they turned to prostitution to help feed their children. " everything is for the children, " one woman says. [SEP] [CLS] tomas medina caracas was a fugitive from a u. s. drug trafficking indictment. " el negro acacio " allegedly helped manage extensive cocaine network. u. s. justice department indicted him in 2002. colombian military : he was killed in an attack on a guerrilla encampment. [SEP

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] harry [MASK]ter star [MASK] hopscotchcliffe gets £20m fortune as he turns 18 monday [MASK] young [MASK] says he has no [MASK] to fritter his cash al9m. radcliffe'[MASK] earnings from firstbaum potter films have been held shishir trust fund. [SEP] [CLS] mentally ill [MASK]mates in miam [MASK] are housed on the " forgotten floor [MASK] [MASK] steven leifman says most are onebridge [MASK] a [MASK] of " avoidable felonies " while cnn tour [MASK] facility, patient shouts : " i intevydis the son of the president " leifman says the system is unjust 8h he's fighting for [MASK]. [SEP] [CLS] new [MASK] " i thought [MASK] was [MASK] to die, " driver says. man says pickup truck [MASK] [MASK]ed in half ; he just has cut on face. driver [MASK] " i probably had a 30 -, [MASK] - [MASK] free fall " minnesota bridge collapsed during rush hour wednesday. [SEP] [CLS] five small polyps found during procedure ; " none worrisome, " spokesman [MASK]. president reclaims powers transferred to vice p

In [ ]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] harry potter star [MASK] radcliffe gets £20m fortune as [MASK] turns 18 monday. [MASK] actor says he has no plans to fritter [MASK] cash away. radcliffe'[MASK] earnings from first five potter [MASK] have been held [MASK] trust fund. [SEP] [CLS] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] miami [MASK] [MASK] on the " forgotten [MASK] [MASK] [MASK] steven leifman says most [MASK] there as a result of [MASK] [MASK] [MASK] felonies " while cnn [MASK] [MASK] facility, [MASK] shouts : " i am the son of the [MASK] " leifman says the system is unjust and he's fighting for [MASK]. [SEP] [CLS] new : [MASK] i thought i was going to [MASK], " driver [MASK]. [MASK] says pickup truck was folded in half [MASK] he just has cut on face. driver : " i probably had a 30 [MASK], [MASK] - foot free fall " minnesota bridge collapsed during [MASK] hour wednesday. [SEP] [CLS] five small [MASK] [MASK] found during [MASK] ; " none worrisome, " spokesman [MASK]. president reclaims powers transferred to 

In [ ]:
train_size = 20_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2000
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    downsampled_dataset["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = model.prepare_tf_dataset(
    downsampled_dataset["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-cnn", tokenizer=tokenizer
)

Cloning https://huggingface.co/viditnaik/SecBERT-finetuned-cnn into local empty directory.


In [ ]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

63/63 [==============================] - 37s 503ms/step - loss: 10.0430
Perplexity: 22993.61


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback])

625/625 [==============================] - 989s 2s/step - loss: 7.1339 - val_loss: 6.3879


In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

63/63 [==============================] - 34s 543ms/step - loss: 6.3779
Perplexity: 588.71
